<a href="https://colab.research.google.com/github/m0hmedtarek/Cell-Images-for-Detecting-Malaria/blob/master/CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://ceb.nlm.nih.gov/proj/malaria/cell_images.zip
 

--2019-04-22 21:54:27--  https://ceb.nlm.nih.gov/proj/malaria/cell_images.zip
Resolving ceb.nlm.nih.gov (ceb.nlm.nih.gov)... 130.14.52.15, 2607:f220:41e:7052::15
Connecting to ceb.nlm.nih.gov (ceb.nlm.nih.gov)|130.14.52.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353452851 (337M) [application/zip]
Saving to: ‘cell_images.zip’

cell_images.zip     100%[===================>] 337.08M  78.1MB/s    in 4.3s    

2019-04-22 21:54:36 (78.2 MB/s) - ‘cell_images.zip’ saved [353452851/353452851]



In [2]:
!unzip cell_images.zip

Archive:  cell_images.zip
   creating: cell_images/
   creating: cell_images/Parasitized/
 extracting: cell_images/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_162.png  
 extracting: cell_images/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_163.png  
 extracting: cell_images/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_164.png  
 extracting: cell_images/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_165.png  
 extracting: cell_images/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_166.png  
 extracting: cell_images/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_167.png  
 extracting: cell_images/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_168.png  
 extracting: cell_images/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_169.png  
 extracting: cell_images/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_170.png  
 extracting: cell_images/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_171.png  
 extracting: cell_images/Parasitized/C100P61ThinF_IMG_201509

In [3]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""

from PIL import Image
import numpy as np
import tensorflow as tf
import os 
from sklearn.model_selection import train_test_split
Image_Size = 64
def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                   if os.path.isdir(os.path.join(data_directory, d))]
    
    labels = []
    images = []
    
    for d in directories:
        
        label_directory = os.path.join(data_directory, d)
        
        file_names = [os.path.join(label_directory, f) 
                      for f in os.listdir(label_directory) 
                      if f.endswith(".png")]
        
        for f in file_names:
            i = Image.open(f).convert('RGB')
            i = i.resize((Image_Size,Image_Size))
            i = np.array(i)
            images.append(i.reshape(Image_Size,Image_Size,3))
            labels.append(d)
    return images, labels



X, y = load_data(os.path.join("cell_images"))

#change 
X = np.array(X).astype(float)
y = np.array(y)

#Encode labels
from sklearn.preprocessing import LabelEncoder 
labellencoder_y = LabelEncoder()
y =labellencoder_y.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X , y , test_size=0.2, random_state=21 , stratify=y)
del X
del y


nnnnnnnnnnnnnnnnn


In [4]:
def cnn_model_fn(features, labels, mode):
  
  #reshaping 
  input_layer = tf.reshape(features, [-1, a, a, 3])

  # Convolutional Layer #1
  #Number of filters 32 and filter size 3*3
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  # MAX_Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  #Number of filters 32 and filter size 3*3
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  # MAX_Pooling Layer #2
  
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatting the shape for The full connected Nueral Network
  
  pool2_flat = tf.reshape(pool2, [-1, 16 * 16 * 32])

  
  #first layer in full connected Nueral Network
  
  dense1 = tf.layers.dense(inputs=pool2_flat, units=512, activation=tf.nn.relu)

 
  dropout1 = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  
  #first layer in full connected Nueral Network
  
  dense2 = tf.layers.dense(inputs=dropout1, units=256, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout2 = tf.layers.dropout(
      inputs=dense2, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  #  layer
  
  logits = tf.layers.dense(inputs=dropout2, units=2 )

  predictions = {
      
      "classes": tf.argmax(input=logits, axis=1),
     
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



  
  # Create the Estimator
mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn)

  # Set up logging for predictions
  
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
      x=X_train,
      y=y_train,
      batch_size=64,
      num_epochs=None,
      shuffle=True)
mnist_classifier.train(
      input_fn=train_input_fn,
      steps=10000,
      hooks=[logging_hook])


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp6kqtblkc', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f525d7ac160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.

In [6]:
#@title evaluate test set { output-height: 150, form-width: "40%", display-mode: "both" }
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=X_test,
    y=y_test,
    num_epochs=1,
    shuffle=True)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-22T22:02:55Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp6kqtblkc/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-22-22:02:57
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.95156026, global_step = 10000, loss = 0.14405242
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: /tmp/tmp6kqtblkc/model.ckpt-10000
{'accuracy': 0.95156026, 'loss': 0.14405242, 'global_step': 10000}


In [5]:
#@title evaluate training set


eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=X_train,
    y=y_train,
    num_epochs=1,
    shuffle=True)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-22T22:02:48Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp6kqtblkc/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-22-22:02:55
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.96058244, global_step = 10000, loss = 0.1168706
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: /tmp/tmp6kqtblkc/model.ckpt-10000
{'accuracy': 0.96058244, 'loss': 0.1168706, 'global_step': 10000}
